In [3]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# load the precomputed embeddings if available

train_embeds = np.load('train_embeds.npy')
val_embeds = np.load('val_embeds.npy')
train_machine_label = np.load('train_machine_label.npy')
val_machine_label = np.load('val_machine_label.npy')
train_binary_label = np.load('train_binary_label.npy')
val_binary_label = np.load('val_binary_label.npy')

In [4]:
train_embeds.shape, \
val_embeds.shape, \
train_machine_label.shape, \
val_machine_label.shape, \
train_binary_label.shape, \
val_binary_label.shape

((39048, 512), (6893, 512), (39048,), (6893,), (39048,), (6893,))

In [5]:
train_embeds_normal = train_embeds[np.where(train_binary_label=='normal')[0]]
train_binary_label_normal = train_binary_label[np.where(train_binary_label=='normal')[0]]
train_machine_label_normal = train_machine_label[np.where(train_binary_label=='normal')[0]]

# Idea 1
<p> if we have both normal and anomaly samples in train foldself. </p>
<p> and Knn neighbour of a test sample is abnormal (or normal) </p>

In [6]:
# Get neighbors from image_embeddings
# fit model
KNN = 15
knn_model = NearestNeighbors(n_neighbors = KNN)
knn_model.fit(train_embeds)
distances, indices = knn_model.kneighbors(val_embeds)

In [7]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [8]:
pred_machine = []
pred_binary = []

for k in range(distances.shape[0]):
    curr_binary = list(train_binary_label[indices[k]])
    curr_machine = list(train_machine_label[indices[k]])
    
    if 'abnormal' in curr_binary[0:5]:
        pred_binary.append('abnormal')
    else:
        pred_binary.append(most_frequent(curr_binary))
        
    pred_machine.append(most_frequent(curr_machine))

In [9]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(val_machine_label, pred_machine))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1633
           1       1.00      1.00      1.00      1572
           2       0.99      0.98      0.99      1585
           3       0.98      0.99      0.99      2103

    accuracy                           0.99      6893
   macro avg       0.99      0.99      0.99      6893
weighted avg       0.99      0.99      0.99      6893



In [10]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(val_machine_label, pred_machine))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1633
           1       1.00      1.00      1.00      1572
           2       0.99      0.98      0.99      1585
           3       0.98      0.99      0.99      2103

    accuracy                           0.99      6893
   macro avg       0.99      0.99      0.99      6893
weighted avg       0.99      0.99      0.99      6893



In [11]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(val_binary_label, pred_binary))

print(classification_report(val_binary_label, pred_binary))

[[1172  111]
 [ 930 4680]]
              precision    recall  f1-score   support

    abnormal       0.56      0.91      0.69      1283
      normal       0.98      0.83      0.90      5610

    accuracy                           0.85      6893
   macro avg       0.77      0.87      0.80      6893
weighted avg       0.90      0.85      0.86      6893



In [12]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(val_binary_label, pred_binary))

print(classification_report(val_binary_label, pred_binary))

[[1172  111]
 [ 930 4680]]
              precision    recall  f1-score   support

    abnormal       0.56      0.91      0.69      1283
      normal       0.98      0.83      0.90      5610

    accuracy                           0.85      6893
   macro avg       0.77      0.87      0.80      6893
weighted avg       0.90      0.85      0.86      6893



# only normal data in train set

<p> Idea 2 </p>
<p> If we have only normal in train fold</p>
<p> then knn neighbour of test sample distance can be used to</p>
<p> judge it as normal or abnormal ?</p>

In [18]:
from sklearn.neighbors import NearestNeighbors
# Get neighbors from image_embeddings
# fit model
KNN = 50
knn_model = NearestNeighbors(n_neighbors = KNN)
knn_model.fit(train_embeds_normal)
distances, indices = knn_model.kneighbors(val_embeds)

In [19]:
pred_machine = []
pred_binary = []
avg_distance = []

for k in range(distances.shape[0]):
    curr_binary = list(train_binary_label_normal[indices[k]])
    curr_machine = list(train_machine_label_normal[indices[k]])
    avg_dist = distances[k][0]#np.mean(distances[k])
    
    #if 'abnormal' in curr_binary:
    #    pred_binary.append('abnormal')
    #else:
    #pred_binary.append(most_frequent(curr_binary))
    pred_machine.append(most_frequent(curr_machine))
    avg_distance.append(avg_dist)

In [20]:
pred_binary = []
thresh = np.mean(avg_distance) + 2*np.std(avg_distance)
for k in avg_distance:
    if k <= thresh:
        pred_binary.append('normal')
    else:
        pred_binary.append('abnormal')

In [21]:
len(pred_binary)

6893

In [22]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(val_machine_label, pred_machine))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1633
           1       1.00      1.00      1.00      1572
           2       0.99      0.98      0.99      1585
           3       0.98      0.99      0.98      2103

    accuracy                           0.99      6893
   macro avg       0.99      0.99      0.99      6893
weighted avg       0.99      0.99      0.99      6893



In [15]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(val_machine_label, pred_machine))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96      1633
           1       1.00      1.00      1.00      1572
           2       0.97      0.94      0.96      1585
           3       0.93      0.97      0.95      2103

    accuracy                           0.96      6893
   macro avg       0.97      0.96      0.97      6893
weighted avg       0.97      0.96      0.96      6893



In [23]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(val_binary_label, pred_binary))

print(classification_report(val_binary_label, pred_binary))

[[ 104 1179]
 [  86 5524]]
              precision    recall  f1-score   support

    abnormal       0.55      0.08      0.14      1283
      normal       0.82      0.98      0.90      5610

    accuracy                           0.82      6893
   macro avg       0.69      0.53      0.52      6893
weighted avg       0.77      0.82      0.76      6893



In [16]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(val_binary_label, pred_binary))

print(classification_report(val_binary_label, pred_binary))

[[  68 1215]
 [ 136 5474]]
              precision    recall  f1-score   support

    abnormal       0.33      0.05      0.09      1283
      normal       0.82      0.98      0.89      5610

    accuracy                           0.80      6893
   macro avg       0.58      0.51      0.49      6893
weighted avg       0.73      0.80      0.74      6893



# Ignore the below stuff if the embeddings are already available

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# Preliminaries
import time
import math
import random
import copy
import shutil
import sys
sys.path.append('../apex')
sys.path.append('../')
import typing as tp
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import collections.abc as container_abcs

from tqdm import tqdm
import yaml
from joblib import delayed, Parallel

# Visuals and CV2
import cv2

import pandas as pd
import numpy as np
from torch.cuda import amp
import matplotlib.pyplot as plt

# albumentations for augs
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

#torch
import torch
import timm
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import Adam, lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler

#from cuml.neighbors import NearestNeighbors

from pytorch_metric_learning import losses

from dataset import SpectrogramDataset


ROOT = Path.cwd()
INPUT_ROOT = ROOT / "../../machine_sound" #"data" / "wav_data"


class CFG:
    seed = 42
    model_name = 'tf_efficientnet_b4_ns' #'efficientnet_b3' #efficientnet_b0-b7
    img_size = 224
    scheduler = 'CosineAnnealingLR'
    T_max = 10
    lr = 1e-5
    min_lr = 1e-6
    batch_size = 16
    weight_decay = 1e-6
    num_epochs = 50
    num_classes = 4
    embedding_size = 512
    n_fold = 0
    n_accumulate = 4
    temperature = 0.1
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
#   torch.backends.cudnn.deterministic = True  # type: ignore
#   torch.backends.cudnn.benchmark = True  # type: ignore


@contextmanager
def timer(name: str) -> None:
    """Timer Util"""
    t0 = time.time()
    print("[{}] start".format(name))
    yield
    print("[{}] done in {:.0f} s".format(name, time.time() - t0))


class SupervisedContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.1):
        super(SupervisedContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, feature_vectors, labels):
        # Normalize feature vectors
        feature_vectors_normalized = F.normalize(feature_vectors, p=2, dim=1)
        # Compute logits
        logits = torch.div(
            torch.matmul(
                feature_vectors_normalized, torch.transpose(feature_vectors_normalized, 0, 1)
            ),
            self.temperature,
        )
        return losses.NTXentLoss(temperature=0.07)(logits, torch.squeeze(labels))

def get_loaders_for_training(
    args_dataset: tp.Dict, args_loader: tp.Dict,
    train_file_list: tp.List[str], val_file_list: tp.List[str]
):
    # # make dataset
    train_dataset = SpectrogramDataset(train_file_list, is_val=False, **args_dataset)
    val_dataset = SpectrogramDataset(val_file_list, is_val=True, **args_dataset)
    # # make dataloader
    train_loader = DataLoader(train_dataset, **args_loader["train"])
    val_loader = DataLoader(val_dataset, **args_loader["val"])

    return train_loader, val_loader, train_dataset, val_dataset


    # 1. Parse dataset and make train/ val folds

tmp_list = []
for decibel_value in INPUT_ROOT.iterdir():
    if decibel_value.is_file():
        continue
    for machine in decibel_value.iterdir():
        if machine.is_file():
            continue
        machine_type = machine.stem
        print(f"Reading files in {machine_type} machine type")
        for id in machine.iterdir():
            if id.is_file():
                continue
            id_type = id.stem
            print(f"Reading files in {id_type}")
            for operation in id.iterdir():
                if operation.is_file():
                    continue
                operation_type = operation.stem
                assert operation_type in ["normal", "abnormal"], "Expected normal or abnormal"
                for wav_f in operation.iterdir():
                    if wav_f.is_file() and wav_f.suffix == ".wav":
                        tmp_list.append( [machine_type, id_type, operation_type,
                                          wav_f.name, wav_f.as_posix()])

train_all = pd.DataFrame(
    tmp_list, columns=["machine_type", "id_type", "operation_type",
                        "wav_filename", "wav_file_path"])

print(train_all.sample(n=5, random_state=1))
print('All df shape ', train_all.shape)

#train_df, val_df = train_test_split(train_all, test_size=0.2, random_state=1234)
train_df, test_df = train_test_split(train_all, test_size=0.15, random_state=1234)
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=1234)

print('Train df shape ', train_df.shape)
print('Test df shape ', val_df.shape)

#TODO: Here we can select to use only normal or abnormal data for training

train_file_list = train_df[["wav_file_path", "machine_type"]].values.tolist()
val_file_list = val_df[["wav_file_path", "machine_type"]].values.tolist()

print("train: {}, val: {}".format(len(train_file_list), len(val_file_list)))

with open('../test_config.yaml') as settings_str:
    settings = yaml.safe_load(settings_str)

for k, v in settings.items():
    print("[{}]".format(k))
    print(v)

set_seed(settings["globals"]["seed"])
device = torch.device(settings["globals"]["device"])
output_dir = Path(settings["globals"]["output_dir"])

# # # get loader
train_loader, valid_loader, train_dataset, valid_dataset = get_loaders_for_training(
    settings["dataset"]["params"], settings["loader"], train_file_list, val_file_list)

dataset_sizes = {
    'train' : train_df.shape[0],
    'valid' : val_df.shape[0]
}
dataloaders = {
    'train' : train_loader,
    'valid' : valid_loader
}

model = timm.create_model(CFG.model_name, pretrained=True)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, CFG.embedding_size)

out = model(torch.randn(1, 3, CFG.img_size, CFG.img_size))
print(f'Embedding shape: {out.shape}')

model.to(CFG.device)

Reading files in pump machine type
Reading files in id_04
Reading files in id_00
Reading files in id_06
Reading files in id_02
Reading files in .ipynb_checkpoints machine type
Reading files in abnormal machine type
Reading files in .ipynb_checkpoints
Reading files in normal machine type
Reading files in slider machine type
Reading files in id_04
Reading files in id_00
Reading files in id_06
Reading files in id_02
Reading files in slider machine type
Reading files in id_04
Reading files in id_00
Reading files in id_06
Reading files in id_02
Reading files in pump machine type
Reading files in id_04
Reading files in id_00
Reading files in id_06
Reading files in id_02
Reading files in pump machine type
Reading files in id_04
Reading files in id_00
Reading files in id_06
Reading files in id_02
Reading files in fan machine type
Reading files in id_04
Reading files in id_00
Reading files in id_06
Reading files in id_02
Reading files in fan machine type
Reading files in id_04
Reading files in 

EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
     

In [3]:
# Load weights
import glob
from collections import defaultdict
load_path = ''
weight_files = glob.glob("../training_output_metricLearning/*.bin")#"#preprocess*")
weight_files.sort()

curr = weight_files[0]
for ctr, k in enumerate(weight_files):
    print(ctr, "\t", weight_files[ctr])
    curr = k 
    
indx = int(input("please enter your selection"))
load_path = weight_files[indx]

model.load_state_dict(torch.load(load_path))
model.eval()

0 	 ../training_output_metricLearning/Model_0.05526390351424902_epoch_37.bin
1 	 ../training_output_metricLearning/Model_0.05826756489623_epoch_36.bin
2 	 ../training_output_metricLearning/Model_0.06430179998008606_epoch_33.bin
3 	 ../training_output_metricLearning/Model_0.06592601374279894_epoch_32.bin
4 	 ../training_output_metricLearning/Model_0.06714329547085465_epoch_26.bin
5 	 ../training_output_metricLearning/Model_0.0710859693136593_epoch_24.bin
6 	 ../training_output_metricLearning/Model_0.08321880577275324_epoch_22.bin
7 	 ../training_output_metricLearning/Model_0.09015285913459173_epoch_21.bin
8 	 ../training_output_metricLearning/Model_0.09345970184241413_epoch_20.bin
9 	 ../training_output_metricLearning/Model_0.10273151750921966_epoch_16.bin
10 	 ../training_output_metricLearning/Model_0.11118020388893003_epoch_12.bin
11 	 ../training_output_metricLearning/Model_0.14495168701305378_epoch_9.bin
12 	 ../training_output_metricLearning/Model_0.14777221272162264_epoch_8.bin
13

EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
     

In [ ]:
############## Inference Using Trained model ###############

all_embeds = []
machine_labels = []
binary_labels = []

for _phase in ['train', 'valid']:
    embeds = []
    machine_label = []
    binary_label = []
    with torch.no_grad():
        phase = _phase
        # Iterate over data
        for inputs,labels in tqdm(dataloaders[phase]):
            inputs = inputs.to(CFG.device)
            #labels = labels.to(CFG.device)
            machine_label.append(labels[0])
            binary_label.append(labels[1])

            feat = model( inputs )
            embeddings = feat.detach().cpu().numpy()
            embeds.append(embeddings)

    image_embeddings = np.concatenate(embeds)
    t1 = np.concatenate(machine_label)
    t2 = np.concatenate(binary_label)
    all_embeds.append(image_embeddings)
    machine_labels.append(t1)
    binary_labels.append(t2)
    print(f'Our image embeddings shape is {image_embeddings.shape}')

 49%|████▊     | 791/1627 [05:52<06:13,  2.24it/s]

In [5]:
train_embeds = all_embeds[0]
val_embeds = all_embeds[1]
train_machine_label = machine_labels[0]
val_machine_label = machine_labels[1]
train_binary_label = binary_labels[0]
val_binary_label = binary_labels[1]

In [6]:
np.save('train_embeds.npy', train_embeds)
np.save('val_embeds.npy', val_embeds)
np.save('train_machine_label.npy', train_machine_label)
np.save('val_machine_label.npy', val_machine_label)
np.save('train_binary_label.npy', train_binary_label)
np.save('val_binary_label.npy', val_binary_label)